MODELIZATION

In [55]:
#!pip install xgboost lightgbm


In [56]:
import pandas as pd
import numpy as np
import time
from utils import general_overview
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

file_path = "..\data\processed-data\madrid_vivienda_sales_processed.parquet"
df = pd.read_parquet(file_path, engine='pyarrow')

file_path3 = "..\data\processed-data\metadata.parquet"
df_meta = pd.read_parquet(file_path3, engine='pyarrow')


In [57]:
general_overview(df)

Column Name                              | Type       | Missing         | Unique     | Sample Values
--------------------------------------------------------------------------------------------------------------
reference_id                             | int64      | 0 (0.0%)        | 59988      | 175162, 315919, 332409
price                                    | float64    | 0 (0.0%)        | 4997       | 795000.0, 475000.0, 180000.0
initial_price                            | float64    | 0 (0.0%)        | 5004       | 795000.0, 475000.0, 180000.0
price_down_from                          | float64    | 57053 (95.1%)   | 983        | 299000.0, 340000.0, 430000.0
price_variation                          | float64    | 0 (0.0%)        | 3941       | 0.0, 0.07777777777777778, 0.00398406374501992
price_volatility                         | float64    | 0 (0.0%)        | 1498       | 0.0, 18027.756377319947, 1414.213562373095
num_price_changes                        | int64      | 0 (0.0%)   

SPLITTING THE DATASET

In [58]:

#We must drop rows that doesn't have the area information.
df_clean = df.dropna(subset=['price_m2']).copy()
target = 'price_m2'

#Columns that contain the answer
leakage_cols = [
    'price', 
    'initial_price', 
    'price_m2', 
    'price_down_from', 
    'price_variation', 
    'price_volatility', 
    'relative_volatility'
]

X = df_clean.drop(columns=leakage_cols, errors='ignore')
y = df_clean[target]


X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=df_clean['loc_cluster']
)


X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=X_temp['loc_cluster']
)


train_stats = X_train.copy()
train_stats['target_price_m2'] = y_train

# Output check
print(f"Training Data:   {X_train.shape[0]} rows ")
print(f"Validation Data: {X_val.shape[0]} rows ")
print(f"Test Data:       {X_test.shape[0]} rows  ")

Training Data:   41835 rows 
Validation Data: 8965 rows 
Test Data:       8965 rows  


WE NEED THE AVERAGE PRICES BASED ON NEIGHBORHOODS (CLUSTERS) IN OUR TRAINING SET

In [59]:
cluster_stats = train_stats.groupby('loc_cluster')['target_price_m2'].agg(['median', 'count'])
global_median = y_train.median()


def calculate_smoothed_benchmark(row, global_val, m=20):
    # m = smoothing factor
    count = row['count']
    local_median = row['median']
    
    # Weighted Average
    smoothed_val = ((count * local_median) + (m * global_val)) / (count + m)
    return smoothed_val


cluster_stats['smoothed_benchmark'] = cluster_stats.apply(
    lambda x: calculate_smoothed_benchmark(x, global_median, m=20), axis=1
)


cluster_map = cluster_stats['smoothed_benchmark'].to_dict()


def apply_benchmark(df, mapping, global_val):
    df['cluster_benchmark_m2'] = df['loc_cluster'].map(mapping)
    
    df['cluster_benchmark_m2'] = df['cluster_benchmark_m2'].fillna(global_val)
    return df

X_train = apply_benchmark(X_train.copy(), cluster_map, global_median)
X_val = apply_benchmark(X_val.copy(), cluster_map, global_median)
X_test = apply_benchmark(X_test.copy(), cluster_map, global_median)


print(f"Global Median: {global_median:.2f}")
print(f"Cluster 41 Raw Median: {cluster_stats.loc[41, 'median']:.2f}")
print(f"Cluster 41 Smoothed:   {cluster_stats.loc[41, 'smoothed_benchmark']:.2f}")

Global Median: 3931.46
Cluster 41 Raw Median: 4498.68
Cluster 41 Smoothed:   4486.88


We audit the clusters to see if they are reliable

In [60]:


cluster_check = train_stats.groupby('loc_cluster')['target_price_m2'].agg(['count', 'median', 'std'])


cluster_check['smoothed_benchmark'] = cluster_check.index.map(cluster_map)


cluster_check['correction_impact'] = cluster_check['smoothed_benchmark'] - cluster_check['median']


print("--- DANGER ZONE AUDIT: Did the Smoothing Work? ---")
print(cluster_check.sort_values('count').head(10)[['count', 'median', 'smoothed_benchmark', 'correction_impact']])


cluster_check['instability_score'] = cluster_check['std'] / cluster_check['median']
print("\n--- CHAOS ZONE AUDIT: How did we handle the instability? ---")
print(cluster_check.sort_values('instability_score', ascending=False).head(5)[['count', 'median', 'smoothed_benchmark', 'correction_impact']])

--- DANGER ZONE AUDIT: Did the Smoothing Work? ---
             count       median  smoothed_benchmark  correction_impact
loc_cluster                                                           
35               8  1052.222222         3108.821116        2056.598894
37              16  2205.362319         3164.305850         958.943531
16              18  1632.467320         2842.463822        1209.996502
31              27  2340.425532         3017.461763         677.036231
48              34  2155.891239         2813.509548         657.618309
27              41  2230.769231         2788.372983         557.603752
39              44  2023.674242         2619.857502         596.183260
33              45  2410.628019         2878.576529         467.948509
15             204  2428.748524         2562.919252         134.170728
44             220  2650.100025         2756.880079         106.780054

--- CHAOS ZONE AUDIT: How did we handle the instability? ---
             count       median  sm

They are more reliable with the smoothed benchmark

MODEL SELECTION

In [61]:

#We setup the data
cols_to_drop = ['reference_id']
X_train_comp = X_train.drop(columns=cols_to_drop, errors='ignore')
X_val_comp = X_val.drop(columns=cols_to_drop, errors='ignore')


models = [
    #These models cannot handle Nan values so we use the Imputer
    ("Ridge (Linear)", make_pipeline(SimpleImputer(strategy='median'), StandardScaler(), Ridge(alpha=1.0))),
    ("Lasso (Linear)", make_pipeline(SimpleImputer(strategy='median'), StandardScaler(), Lasso(alpha=0.1))),
    ("KNN (Distance)", make_pipeline(SimpleImputer(strategy='median'), StandardScaler(), KNeighborsRegressor(n_neighbors=10))),
    ("Decision Tree", make_pipeline(SimpleImputer(strategy='median'), DecisionTreeRegressor(max_depth=10, random_state=42))),
    ("Random Forest", make_pipeline(SimpleImputer(strategy='median'), RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1))),
    ("Neural Network (Fast)", make_pipeline(SimpleImputer(strategy='median'), StandardScaler(), MLPRegressor(hidden_layer_sizes=(50,), max_iter=100, random_state=42))),
    #These models handle Nan values so we use raw data (preferably)
    ("Gradient Boosting", GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)),
    ("XGBoost", XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42, n_jobs=-1)),
    ("LightGBM", LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42, n_jobs=-1, verbose=-1))
]


results = []
print(f"{'Model':<30} | {'MAE':<8} | {'RMSE':<8} | {'R2':<6} | {'Error %':<8}")
print("-" * 80)

for name, model in models:
    try:
        start = time.time()
        
        model.fit(X_train_comp, y_train)
        preds = model.predict(X_val_comp)
        
        mae = mean_absolute_error(y_val, preds)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        r2 = r2_score(y_val, preds)
        error_pct = (mae / y_val.mean()) * 100
        elapsed = time.time() - start
        
        results.append({
            "Model": name, 
            "MAE": mae, 
            "RMSE": rmse, 
            "R2 Score": r2, 
            "Error %": error_pct,
            "Time (s)": elapsed
        })
        print(f"{name:<30} | {mae:<8.0f} | {rmse:<8.0f} | {r2:<6.3f} | {error_pct:<7.2f}%")
        
    except Exception as e:
        print(f"{name:<30} | FAILED: {str(e)}")


leaderboard = pd.DataFrame(results).sort_values(by="R2 Score", ascending=False)
print("\n--- FINAL STANDINGS ---")
print(leaderboard[['Model', 'R2 Score', 'MAE', 'Error %', 'Time (s)']])


Model                          | MAE      | RMSE     | R2     | Error % 
--------------------------------------------------------------------------------
Ridge (Linear)                 | 1068     | 1590     | 0.701  | 22.27  %
Lasso (Linear)                 | 1068     | 1590     | 0.701  | 22.27  %
KNN (Distance)                 | 1070     | 1585     | 0.703  | 22.32  %
Decision Tree                  | 886      | 1393     | 0.771  | 18.48  %
Random Forest                  | 804      | 1231     | 0.821  | 16.77  %


c:\Users\juanv\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural Network (Fast)          | 1041     | 1560     | 0.712  | 21.72  %
Gradient Boosting              | FAILED: Input X contains NaN.
GradientBoostingRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values
XGBoost                        | 740      | 1156     | 0.842  | 15.43  %
LightGBM                       | 763      | 1185     | 0.834  | 15.91  %

--- FINAL STANDINGS ---
                   Model  R2 Score          MAE    Error %   Time (

We choose XGBoost

HYPERPARAMETER TUNING

In [62]:


print("--- STARTING OPTIMIZATION FOR XGBOOST ---")


param_grid = {
    'n_estimators': [500, 1000, 1500],        
    'max_depth': [4, 6, 8, 10],               
    'learning_rate': [0.01, 0.05, 0.1],       
    'subsample': [0.6, 0.8, 1.0],             
    'colsample_bytree': [0.6, 0.8, 1.0],      
    'min_child_weight': [1, 5, 10]            
}

# Base Model
xgb_model = XGBRegressor(random_state=42, n_jobs=-1)

# we setup randomized search
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=20,              
    scoring='neg_mean_absolute_error',
    cv=3,                   
    verbose=1,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_comp, y_train)

# We get the best model
best_model = random_search.best_estimator_

print("\n--- TUNING COMPLETE ---")
print(f"Best Parameters: {random_search.best_params_}")

# Final Validation Check
# We compare the optimized model vs. the default one
tuned_preds = best_model.predict(X_val_comp)
tuned_mae = mean_absolute_error(y_val, tuned_preds)
tuned_r2 = r2_score(y_val, tuned_preds)
tuned_error_pct = (tuned_mae / y_val.mean()) * 100

print(f"\n--- FINAL SCORECARD ---")
print(f"Tuned MAE:      €{tuned_mae:.0f}/m2")
print(f"Tuned Error %:  {tuned_error_pct:.2f}% (Previous: 15.74%)")
print(f"Tuned R2 Score: {tuned_r2:.4f}   (Previous: 0.8259)")

--- STARTING OPTIMIZATION FOR XGBOOST ---
Fitting 3 folds for each of 20 candidates, totalling 60 fits

--- TUNING COMPLETE ---
Best Parameters: {'subsample': 0.6, 'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 10, 'learning_rate': 0.01, 'colsample_bytree': 0.8}

--- FINAL SCORECARD ---
Tuned MAE:      €643/m2
Tuned Error %:  13.40% (Previous: 15.74%)
Tuned R2 Score: 0.8686   (Previous: 0.8259)


VALIDATION OF THE MODEL ON THE TEST SET

In [71]:
#Prediction
cols_to_drop = 'reference_id'
X_test_clean = X_test.drop(columns=cols_to_drop, errors='ignore')


predicted_fair_value_m2 = best_model.predict(X_test_clean)


results = X_test[['reference_id']].copy()

# We add predictions and real value to results
results['AI_Fair_Value_m2'] = predicted_fair_value_m2
results['Listed_Price_m2'] = y_test # Pandas alinea esto automáticamente

cols_needed = ['reference_id', 'price', 'price_down_from', 'loc_cluster', 'area']
results = results.merge(df[cols_needed], on='reference_id', how='left')


results.rename(columns={
    'loc_cluster': 'Cluster', 
    'price': 'Total_Price', 
    'area': 'Area_m2'
}, inplace=True)


# We compute the price using the model
results['AI_Total_Fair_Value'] = results['AI_Fair_Value_m2'] * results['Area_m2']

# We calculate benefit
results['Diff_Pct'] = (results['AI_Fair_Value_m2'] - results['Listed_Price_m2']) / results['AI_Fair_Value_m2']


realistic_gems = results[
    (results['Listed_Price_m2'] > 1000) &            # SAFETY FLOOR
    (results['Diff_Pct'] < 0.50) &                   # SAFETY CEILING
    (results['Diff_Pct'] > 0.10) &                   # TARGET
    (results['price_down_from'] > results['Total_Price']) # SIGNAL (Seller lowering price)
].sort_values(by='Diff_Pct', ascending=False).copy()


preds_test = best_model.predict(X_test_clean)

# Metrics
mae = mean_absolute_error(y_test, preds_test)
rmse = np.sqrt(mean_squared_error(y_test, preds_test))
r2 = r2_score(y_test, preds_test)
error_pct = (mae / y_test.mean()) * 100


print(f"--- PERFORMANCE OF THE MODEL IN THE TEST SET ---")
print(f"MAE (Mean Abslute Error):  {mae:.2f} €/m2")
print(f"RMSE (Root Mean Squared Error):{rmse:.2f} €/m2")
print(f"R2 Score:{r2:.4f}")
print(f"Error percentage:{error_pct:.2f}%")


# 5. REPORTE FINAL (Idéntico al tuyo)
print(f"\n--- REALISTIC INVESTMENT STRATEGY REPORT ---")
print(f"Total Properties Scanned (Test Set): {len(X_test)}")
print(f"Valid 'Gems' Found:                  {len(realistic_gems)}")
print(f"Average Potential Profit:            {realistic_gems['Diff_Pct'].mean()*100:.2f}%")

print("\n--- TOP 20 'MUST BUY' PROPERTIES (Cleaned) ---")
# Las columnas existen gracias al rename anterior, así que este print funciona perfecto
print(realistic_gems[['reference_id', 'Total_Price', 'AI_Total_Fair_Value', 'Cluster', 'Listed_Price_m2', 'AI_Fair_Value_m2', 'Diff_Pct']].head(20))

--- PERFORMANCE OF THE MODEL IN THE TEST SET ---
MAE (Mean Abslute Error):  632.63 €/m2
RMSE (Root Mean Squared Error):1035.76 €/m2
R2 Score:0.8656
Error percentage:13.33%

--- REALISTIC INVESTMENT STRATEGY REPORT ---
Total Properties Scanned (Test Set): 8965
Valid 'Gems' Found:                  144
Average Potential Profit:            20.24%

--- TOP 20 'MUST BUY' PROPERTIES (Cleaned) ---
      reference_id  Total_Price  AI_Total_Fair_Value  Cluster  \
5388     105440324      58685.0         1.160139e+05       47   
3677     105353879     131813.0         2.421485e+05       43   
6232      97786120     139900.0         2.565240e+05       23   
5551     103370727     990000.0         1.774771e+06        3   
1187     105126080     124000.0         2.176417e+05        2   
2003     105158524     255000.0         4.175197e+05       34   
5273     103529273     225000.0         3.604673e+05       22   
4995     103645767     330000.0         5.212734e+05       36   
1237     103895507    

WE MERGE THE IDs THAT WE HAVE FOUND WITH THE METADATA INFO IN ORDER TO HAVE A COMPLETE DESCRIPTION OF THE CASES.

In [ ]:

realistic_gems['reference_id'] = realistic_gems['reference_id'].astype(str).str.strip()
df_meta['reference_id'] = df_meta['reference_id'].astype(str).str.strip()


cols_meta = ['reference_id', 'ad_url', 'address', 'ad_description']
meta_unique = df_meta[cols_meta].drop_duplicates(subset='reference_id', keep='first')


gems_enriched = realistic_gems.merge(meta_unique, on='reference_id', how='left')


gems_enriched = gems_enriched.drop_duplicates(subset='reference_id', keep='first')



# Final report
print(f"\n{'='*80}")
print(f"--- DETAILED DESCRIPTIONS OF TOP ({len(gems_enriched)} Gems) ---")
print(f"{'='*80}\n")

for index, row in gems_enriched.head(20).iterrows(): 
    
    # Financial 
    ref = row['reference_id']
    precio = row['Total_Price']
    valor_ai = row['AI_Total_Fair_Value']
    beneficio = row['Diff_Pct'] * 100
    
    # Descriptive
    url = row['ad_url'] if pd.notna(row['ad_url']) else "No URL"
    direccion = row['address'] if pd.notna(row['address']) else "No direction"
    desc = row['ad_description'] if pd.notna(row['ad_description']) else "No description"
    
    
    desc_limpia = str(desc).replace('<br>', '\n').strip()
    if len(desc_limpia) > 300:
        desc_limpia = desc_limpia[:300] + " [...]"

    print(f"🆔 REF: {ref}")
    print(f"📍 ADDRESS:   {direccion}")
    print(f"🔗 URL:         {url}")
    print(f"💰 ASKING PRICE:   {precio:,.0f} €")
    print(f"🤖 IA FAIR PRICE: {valor_ai:,.0f} €")
    print(f"📈 STAT. PROFIT: {beneficio:.2f}%")
    print("-" * 30)
    print(f"📝 {desc_limpia}")
    print(f"\n{'-'*80}\n")


--- DETAILED DESCRIPTIONS OF TOP (144 Gems) ---

🆔 REF: 105440324
📍 DIRECTION:   Calle Soldado Jose Maria Rey
🔗 URL:         https://www.idealista.com/inmueble/105440324/
💰 PRICE:   58,685 €
🤖 IA FAIR PRICE: 116,014 €
📈 BENEFIT : 49.42%
------------------------------
📝 Oportunidad Unica de Inversion - Activo Inmobiliario REFERENCIA: 25264[Auctree. com/auction/25264] Hoy os presentamos una excelente oportunidad de inversion que ofrece un potencial significativo de revalorizacion y rentabilidad.Auctree es la plataforma lider en Espana para inversores inmobiliarios.  [...]

--------------------------------------------------------------------------------

🆔 REF: 105353879
📍 DIRECTION:   Calle de la Reguera de Tomateros
🔗 URL:         https://www.idealista.com/inmueble/105353879/
💰 PRICE:   131,813 €
🤖 IA FAIR PRICE: 242,149 €
📈 BENEFIT : 45.57%
------------------------------
📝 Oportunidad Unica de Inversion - Activo Inmobiliario REFERENCIA: 24773[Auctree. com/auction/24773] Hoy os present

WE RUN THE ALGORYTHM ON THE TRAINING SET TO FIND OTHER GEMS

In [74]:
print("\n" + "-"*80)
print("--- ANALYZING TRAINING SET ---")
print("-"*80 + "\n")

# PREDICTION

cols_drop_model = ['reference_id', 'id']
X_train_clean = X_train.drop(columns=cols_drop_model, errors='ignore')

predicted_fair_value_train = best_model.predict(X_train_clean)

# BASE TABLE CONSTRUCTION 
results_train = X_train[['reference_id']].copy()
results_train['AI_Fair_Value_m2'] = predicted_fair_value_train
results_train['Listed_Price_m2'] = y_train 

#DATA RECOVERY FROM ORIGINAL DF 
cols_needed = ['reference_id', 'price', 'price_down_from', 'loc_cluster', 'area']
results_train = results_train.merge(df[cols_needed], on='reference_id', how='left')

# Rename to maintain naming standard
results_train.rename(columns={
    'loc_cluster': 'Cluster', 
    'price': 'Total_Price', 
    'area': 'Area_m2'
}, inplace=True)

# FINANCIAL CALCULATIONS
results_train['AI_Total_Fair_Value'] = results_train['AI_Fair_Value_m2'] * results_train['Area_m2']
results_train['Diff_Pct'] = (results_train['AI_Fair_Value_m2'] - results_train['Listed_Price_m2']) / results_train['AI_Fair_Value_m2']

# GEM FILTERING 
gems_train = results_train[
    (results_train['Listed_Price_m2'] > 1000) &            
    (results_train['Diff_Pct'] < 0.50) &                   
    (results_train['Diff_Pct'] > 0.10) &                   
    (results_train['price_down_from'] > results_train['Total_Price']) 
].sort_values(by='Diff_Pct', ascending=False).copy()

# ==========================================
# METADATA ENRICHMENT (NO DUPLICATES)
# ==========================================

# Standardize IDs to string
gems_train['reference_id'] = gems_train['reference_id'].astype(str).str.strip()
df_meta['reference_id'] = df_meta['reference_id'].astype(str).str.strip()

# Clean Duplicates in Metadata
cols_meta = ['reference_id', 'ad_url', 'address', 'ad_description']
meta_unique = df_meta[cols_meta].drop_duplicates(subset='reference_id', keep='first')

# Final Merge
gems_train_full = gems_train.merge(meta_unique, on='reference_id', how='left')

# ==========================================
# FINAL REPORT (TRAINING SET)
# ==========================================

print(f"--- TRAINING SET OPPORTUNITIES REPORT ---")
print(f"Total Properties Scanned (Train Set): {len(X_train)}")
print(f"Valid 'Gems' Found:                   {len(gems_train_full)}")
print(f"Average Potential Profit:             {gems_train_full['Diff_Pct'].mean()*100:.2f}%")
print(f"{'='*80}\n")

# Show TOP 20
for index, row in gems_train_full.head(20).iterrows():
    
    # Data
    ref = row['reference_id']
    precio = row['Total_Price']
    valor_ai = row['AI_Total_Fair_Value']
    beneficio = row['Diff_Pct'] * 100
    
    # Safe text retrieval
    url = row['ad_url'] if pd.notna(row['ad_url']) else "No URL"
    direccion = row['address'] if pd.notna(row['address']) else "Address not available"
    desc = row['ad_description'] if pd.notna(row['ad_description']) else "No description"
    
    # Visual cleanup
    desc_limpia = str(desc).replace('<br>', '\n').strip()
    if len(desc_limpia) > 300: desc_limpia = desc_limpia[:300] + " [...]"

    print(f"🆔 REF: {ref} (TRAIN)")
    print(f"📍 ADDRESS:        {direccion}")
    print(f"🔗 URL:            {url}")
    print(f"💰 ASKING PRICE:   {precio:,.0f} €")
    print(f"🤖 AI FAIR VALUE:  {valor_ai:,.0f} €")
    print(f"📈 STAT. PROFIT:    {beneficio:.2f}%")
    print("-" * 30)
    print(f"📝 {desc_limpia}")
    print(f"\n{'-'*80}\n")


--------------------------------------------------------------------------------
--- ANALYZING TRAINING SET ---
--------------------------------------------------------------------------------

--- TRAINING SET OPPORTUNITIES REPORT ---
Total Properties Scanned (Train Set): 41835
Valid 'Gems' Found:                   348
Average Potential Profit:             17.44%

🆔 REF: 26844656 (TRAIN)
📍 ADDRESS:        Calle de San Agapito
🔗 URL:            https://www.idealista.com/inmueble/26844656/
💰 ASKING PRICE:   62,000 €
🤖 AI FAIR VALUE:  116,188 €
📈 STAT. PROFIT:    46.64%
------------------------------
📝 SOLO INVERSORES: ALQUILADA HASTA EL 1/1/2026 CUOTA 126 EUROS LA MES. NO SE PUEDE VISITAR, SOLO SE PAGA AL CONTADO SIN HIPOTECAS. Dispone de buenas comunicaciones tanto por transporte publico (a poca distancia de estacion de metro de Villaverde Bajo-Cruce, y del cercanias de Villaverde, asi como vari [...]

--------------------------------------------------------------------------------

🆔

In [70]:
# ==============================================================================
# GENERATING THE FINAL INVESTMENT DOSSIER (Test + Train)
# ==============================================================================

# 1. Ensure both tables are ready (Test and Train)
# (Assuming previous blocks were executed, gems_enriched and gems_train_full exist in memory)

# Add a column to identify the source of each opportunity
gems_enriched['Origen_Datos'] = 'TEST SET (External Validation)'
gems_train_full['Origen_Datos'] = 'TRAINING SET (High Confidence Opportunity)'

# 2. Merge both tables
all_gems = pd.concat([gems_enriched, gems_train_full], ignore_index=True)

# 3. Select and order ONLY the columns useful for the client
cols_export = [
    'reference_id',       # ID
    'Origen_Datos',       # Test or Train Source
    'address',            # Address
    'Total_Price',        # Actual Listed Price
    'AI_Total_Fair_Value',# Our AI Valuation
    'Diff_Pct',           # Profit Margin %
    'Area_m2',            # Square Meters
    'ad_description',     # Description
    'ad_url'              # Web Link
]

# Create the final clean DataFrame
final_dossier = all_gems[cols_export].copy()

# 4. Final Formatting and Cleaning for Excel
# Round prices
final_dossier['Total_Price'] = final_dossier['Total_Price'].round(0)
final_dossier['AI_Total_Fair_Value'] = final_dossier['AI_Total_Fair_Value'].round(0)

# Convert % to readable format (0.25 -> 25.0)
final_dossier['Diff_Pct'] = (final_dossier['Diff_Pct'] * 100).round(2)

# Rename columns to Professional English for Rapada Partners
final_dossier.rename(columns={
    'reference_id': 'Reference_ID',
    'Origen_Datos': 'Source',
    'address': 'Address',
    'Total_Price': 'Current_Ask_Price (€)',
    'AI_Total_Fair_Value': 'AI_Fair_Value (€)',
    'Diff_Pct': 'Potential_Profit (%)',
    'Area_m2': 'Surface_Area (m2)',
    'ad_description': 'Description',
    'ad_url': 'Web_Link'
}, inplace=True)

# Sort by highest potential profit
final_dossier = final_dossier.sort_values(by='Potential_Profit (%)', ascending=False)

# 5. SAVE FILE
output_file = "RAPADA_PARTNERS_Opportunity_Dossier.xlsx"

# Use try/except in case openpyxl is not installed (otherwise saves as CSV)
try:
    final_dossier.to_excel(output_file, index=False)
    print(f"✅ Excel file generated successfully: {output_file}")
except ImportError:
    final_dossier.to_csv("RAPADA_PARTNERS_Opportunity_Dossier.csv", index=False, sep=';')
    print(f"⚠️ Excel library not detected. Saved as CSV: RAPADA_PARTNERS_Opportunity_Dossier.csv")

print(f"Total Opportunities Exported: {len(final_dossier)}")
print("First 5 rows of the dossier:")
display(final_dossier.head())

✅ Excel file generated successfully: RAPADA_PARTNERS_Opportunity_Dossier.xlsx
Total Opportunities Exported: 492
First 5 rows of the dossier:


,Reference_ID,Source,Address,Current_Ask_Price (€),AI_Fair_Value (€),Potential_Profit (%),Surface_Area (m2),Description,Web_Link
0,105440324,TEST SET (External Validation),Calle Soldado Jose Maria Rey,58685.0,116014.0,49.42,52.0,Oportunidad Unica de Inversion - Activo Inmobi...,https://www.idealista.com/inmueble/105440324/
144,26844656,TRAINING SET (High Confidence Opportunity),Calle de San Agapito,62000.0,116188.0,46.64,60.0,SOLO INVERSORES: ALQUILADA HASTA EL 1/1/2026 C...,https://www.idealista.com/inmueble/26844656/
1,105353879,TEST SET (External Validation),Calle de la Reguera de Tomateros,131813.0,242149.0,45.57,124.0,Oportunidad Unica de Inversion - Activo Inmobi...,https://www.idealista.com/inmueble/105353879/
2,97786120,TEST SET (External Validation),Calle de los Hermanos Garcia Noblejas,139900.0,256524.0,45.46,85.0,Planta novena con ascensor. Muy buena zona. So...,https://www.idealista.com/inmueble/97786120/
145,104459486,TRAINING SET (High Confidence Opportunity),Calle 30,96511.0,176348.0,45.27,58.0,Oportunidad de inversion. Propiedad disponible...,https://www.idealista.com/inmueble/104459486/
